In [1]:
import os
import re
import cv2 # opencv library
import numpy as np
from os.path import isfile, join
import matplotlib.pyplot as plt

In [52]:
path = os.getcwd()+"\\object_detection\\input\\"
name = "traffic.mp4"
vidObj = cv2.VideoCapture(path+name)
frames = []
frames_processed = []
success = 1
kernel = np.ones((4,4),np.uint8)
font = cv2.FONT_HERSHEY_SIMPLEX
while success:
    success, image = vidObj.read()
    frames.append(image)
# print(len(frames))
for i in range(len(frames)-2):
    grayA = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(frames[i+1], cv2.COLOR_BGR2GRAY)
    diff_image = cv2.absdiff(grayB, grayA)
    height, width = diff_image.shape
    min_x, min_y = width, height
    max_x = max_y = 0
    ret, thresh = cv2.threshold(diff_image, 30, 255, cv2.THRESH_BINARY)  
    dilated = cv2.dilate(thresh,kernel,iterations = 1)  
    contours, hierarchy = cv2.findContours(dilated.copy(), cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    valid_cntrs = []
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
        if (x <= width) & (height >= int(height/2)) & (cv2.contourArea(cntr) >= 2500) & (cv2.contourArea(cntr) <= 5000):
            if (y >= 90) & (cv2.contourArea(cntr) < 40):
                break
            valid_cntrs.append(cntr)
    dmy = frames[i].copy()
    #cv2.drawContours(dmy, valid_cntrs, -1, (127,200,0), 2)
    cv2.putText(dmy, "vehicles detected: " + str(len(valid_cntrs)), (55, 15), font, 0.6, (0, 180, 0), 2)
    cv2.line(dmy, (0, int(height/2)),(int(width),int(height/2)),(100, 255, 255),3)
    for cntr in valid_cntrs:
        x,y,w,h = cv2.boundingRect(cntr)
        min_x, max_x = min(x, min_x), max(x+w, max_x)
        min_y, max_y = min(y, min_y), max(y+h, max_y)
        cv2.rectangle(dmy, (x,y), (x+w,y+h), (255, 0, 0), 2)
    height, width, layers = dmy.shape
    size = (width,height)
    frames_processed.append(dmy)
# len(frames_processed)

In [53]:
pathOut = os.getcwd()+"\\object_detection\\output\\"+'vehicle_detection.mp4'
fps = 7
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
for i in range(len(frames_processed)):
    out.write(frames_processed[i])
out.release()

In [54]:
cv2.destroyAllWindows() 